### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [2]:
dataalbums = pd.read_csv('../data/clean/api_albums_clean.csv', encoding='latin1')

In [3]:
dataalbums.head(2)

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,6QaVfG1pHYl1z15ZxkvVDW,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1967-06-01,13.0
1,Pet Sounds,6GphKx2QAPRoVGWE9D7ou8,Pet Sounds (Original Mono & Stereo Mix),The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-06-16,27.0


In [4]:
artist_ids = dataalbums['artistidspotify'].unique().tolist()

In [5]:
artist_ids[0:2]

['3WrFJ7ztbogyGnTHbHJFl2', '3oDbviiivRWhXwIE8hxkVV']

### Spotify Connection 

In [6]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [7]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [8]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Artist Information 

In [9]:
artist_ids = dataalbums['artistidspotify'].unique().tolist()

In [10]:
artist_ids[0:3]

['3WrFJ7ztbogyGnTHbHJFl2', '3oDbviiivRWhXwIE8hxkVV', '74ASZWbe4lXaubB36ztrGX']

In [11]:
artist_urls = [f'https://api.spotify.com/v1/artists/{artist_id}' for artist_id in artist_ids]

artist_url_list = [{'id': artist_id, 'url': url} for artist_id, url in zip(artist_ids, artist_urls)]

In [12]:
# List to store the retrieved artist data
artist_data_list = []

# Function to search and retrieve artist information
def search_artist_info(artist_url):
    time.sleep(0.30)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(artist_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        artist_data_list.append({
            'artistidspotify': artist_url['id'],  # Spotify ID for the artist
            'artistnamespotify': data.get('name', None),
            'followers': data.get('followers', {}).get('total', None),
            'popularity': data.get('popularity', None),
            'genres': data.get('genres', None)
            # Add more data points as needed
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        artist_data_list.append({
            'artistidspotify': artist_url['id'],
            'artistnamespotify': None,
            'followers': None,
            'popularity': None,
            'genres': None
            # Add more data points as needed
        })

# Execute the function for each artist URL
for artist_url in artist_url_list:
    search_artist_info(artist_url)

# Create a DataFrame from the list of dictionaries
dataartists = pd.DataFrame(artist_data_list)

An error occurred: 400 Client Error: Bad Request for url: https://api.spotify.com/v1/artists/nan


In [14]:
dataartists.head(5)

,artistidspotify,artistnamespotify,followers,popularity,genres
0,3WrFJ7ztbogyGnTHbHJFl2,The Beatles,26469550.0,82.0,"[beatlesque, british invasion, classic rock, m..."
1,3oDbviiivRWhXwIE8hxkVV,The Beach Boys,4308475.0,68.0,"[baroque pop, classic rock, folk rock, mellow ..."
2,74ASZWbe4lXaubB36ztrGX,Bob Dylan,6291947.0,68.0,"[classic rock, country rock, folk, folk rock, ..."
3,3koiLjNrgRTNbOwViDipeA,Marvin Gaye,5189557.0,71.0,"[classic soul, motown, neo soul, northern soul..."
4,22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,13455698.0,78.0,"[album rock, british invasion, classic rock, r..."


In [15]:
dataartists.to_csv('../data/clean/api_artists_clean.csv', index=False)